# 1. Crawl data

In [45]:
import pandas as pd
from selenium import webdriver
import time


def crawl(driverfile = '/home/phamhung/Documents/crawl_data/selenium/chromedriver', url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'):
    driver = webdriver.Chrome(driverfile)
    driver.get(url)
    time.sleep(2)
    listneighbor = driver.find_elements_by_xpath('//table[@class="wikitable sortable jquery-tablesorter"]/tbody/tr')

    neighbor = pd.DataFrame(columns=['Postal Code','Borough','Neighborhood'])
    for i in range(len(listneighbor)):
        neighbor.loc[i,'Postal Code'] = eval(f'driver.find_element_by_xpath(\'//table[@class="wikitable sortable jquery-tablesorter"]/tbody/tr[{i+1}]/td[1]\').text')
        neighbor.loc[i,'Borough'] = eval(f'driver.find_element_by_xpath(\'//table[@class="wikitable sortable jquery-tablesorter"]/tbody/tr[{i+1}]/td[2]\').text')
        neighbor.loc[i,'Neighborhood'] = eval(f'driver.find_element_by_xpath(\'//table[@class="wikitable sortable jquery-tablesorter"]/tbody/tr[{i+1}]/td[3]\').text')
    return neighbor

In [51]:
neighbor = crawl()
neighbor = neighbor[neighbor.Borough != 'Not assigned']
neighbor.Neighborhood.replace('Not assigned',neighbor.Borough, inplace = True)
print('Shape: ',neighbor.shape)

# 2. Location of neighborhoods

In [78]:
latlng = pd.read_csv('/home/phamhung/Documents/data_engineer/Coursera_Capstone/Geospatial_Coordinates.csv')
latlng

neighbor = neighbor.merge(latlng, how='left', on = 'Postal Code')



,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# 3. Clustering
